# Get Segmented Histogram

This notebook demonstrates how to:
1. Import raw grayscale CT/microCT data
2. Generate and visualize histograms of intensity distributions based on the threshold segmentation.
3. Save the colored histogram figure to the output folder

## Import Required Modules

We'll use the Digital Rock Physics template package to handle data import, visualization, and analysis:

In [ ]:
import drp_template.io as io
import drp_template.image as image
import drp_template.default_params as params
import drp_template.compute as compute

## Load Grayscale Data

Set the file path and dimensions for the raw uint16 data file. This example uses a 50³ voxel cube of segmented CT data.

**Parameters:**
- `raw_file_path`: Path to the raw binary file containing grayscale intensity values
- `dimensions`: Dictionary specifying the 3D dimensions (nz, ny, nx) of the dataset
- `dtype`: Data type of the raw file ('uint16' for 16-bit unsigned integer data)

In [ ]:
# Set file paths and dimensions
raw_file_path = '../../data/data_raw_uint16_50cube.raw'
dimensions = {'nz': 50, 'ny': 50, 'nx': 50}

voxel_size_um = 1.0  # Voxel size in micrometers
voxel_size_m = compute.um2m(voxel_size_um)  # Convert voxel size to meters

# Import the raw data
# This will automatically generate a JSON parameter file with the same base name
data = io.import_model(file_path=raw_file_path, voxel_size=voxel_size_m, dtype='uint16', dimensions=dimensions)
# The parameter filename follows the naming convention: <base_filename>.json
# This file stores metadata about dimensions, data type, and analysis parameters

## Define the Parameter File

After importing the data, a JSON parameter file is automatically created. This file stores metadata about your dataset including:
- Dimensions (nx, ny, nz)
- Data type (uint16, uint8, etc.)
- File paths
- Voxel size (in meters, SI units)
- Analysis parameters (thresholds, labels, etc.)

**Naming Convention:** 
By default, the parameter file is named `parameters.json`. You can customize this using the `paramsfile` argument:
```python
data = io.import_model(..., paramsfile='data_raw_uint16_50cube.json')
```

This flexibility allows you to:
- Use descriptive names matching your raw data files
- Manage multiple parameter files in the same directory
- Avoid filename collisions when working with multiple datasets

## Define Threshold Ranges for Colored Histogram

To visualize the grayscale histogram with color-coded phases, we define threshold ranges that correspond to different material phases. These thresholds represent the grayscale intensity ranges that were used during the segmentation process.

**Threshold Dictionary Structure:**
- Each key represents a material phase name
- Each value is a list `[min_intensity, max_intensity]` defining the grayscale range for that phase

**Example Phases:**
- **Pore**: Low-density voids (darkest in grayscale images)
- **Qtz**: Quartz mineral phase
- **Fsp**: Feldspar mineral phase
- **Bt**: Biotite mineral phase
- **hDensity**: High-density minerals (brightest in grayscale images)

The thresholds are saved to the parameter file for documentation and reproducibility.

In [ ]:
# Define threshold ranges for different material phases
thresholds = {
    "Pore": [0, 4000],
    "Qtz": [4001, 4550],
    "Fsp": [4551, 5500],
    "Bt": [5501, 11000],
    "hDensity": [11001, 65535]
}

# Save thresholds to parameter file
params.update_parameters_file(paramsfile='parameters.json', thresholds=thresholds)

# Display the defined threshold ranges
print("Threshold ranges defined:")
for phase, range_vals in thresholds.items():
    print(f"  {phase:10s}: {range_vals[0]:5d} - {range_vals[1]:5d}")

## Visualize Colored Histogram

Generate a histogram of the grayscale intensity distribution with color-coded regions corresponding to the defined threshold ranges. Each material phase will be displayed in a different color, making it easy to visualize how the grayscale values are distributed across different materials.

**Histogram Parameters:**
- `data`: The 3D grayscale array to analyze
- `thresholds`: Dictionary of phase ranges for color-coding
- `paramsfile`: JSON file containing metadata
- `log_scale='y'`: Use logarithmic scale on y-axis for better visualization of low-frequency bins

**Output:**
The histogram will be saved as `figure_colored_hist.png` in the `output/` folder, showing the intensity distribution with each phase highlighted in a distinct color.

In [ ]:
fig, ax = image.histogram(
    data=data,
    thresholds=thresholds,
    paramsfile='parameters.json',
    dtype=None,
    cmap_set=None,
    title=None,
    log_scale='y',
    dark_mode=False,
    num_bins=None
)

image.save_figure(fig, filename='figure_colored_hist')

## Get the Used Color Codes

Extract the color palette from the histogram for documentation, publication, or design purposes. This is particularly useful when you need to:

- **Report colors in manuscripts**: Include exact RGB/CMYK values in figure captions
- **Reproduce colors in other software**: Use HEX codes in presentations, web pages, or design tools
- **Maintain color consistency**: Ensure the same colors across multiple figures or publications
- **Print preparation**: Convert RGB colors to CMYK for professional printing

The `get_figure_colors()` function distinguishes between:
- **Data colors** (colormap colors from the histogram bars) - extracted by default
- **Decoration colors** (axes, text, background) - optional for complete figure documentation

**Output Formats:**
- **RGB**: Values from 0 to 1 (standard for Python/Matplotlib)
- **CMYK**: Values from 0 to 100% (standard for print publications)
- **HEX**: Format `#RRGGBB` (standard for web design and presentations)

In [ ]:
# Extract only colormap/data colors (default behavior)
colors = image.get_figure_colors(fig, num_colors=10, data_only=True)

# Access data colors in different formats
data_rgb = colors['rgb']['data_colors']        # List of RGB tuples (0-1 range)
data_cmyk = colors['cmyk']['data_colors']      # List of CMYK tuples (0-100% range)
data_hex = colors['hex']['data_colors']        # List of HEX strings

print(f"\nExtracted {len(data_hex)} colormap colors")
print("First 3 HEX codes:", data_hex[:3])

In [ ]:
# Get all colors including axes, text, background colors
all_colors = image.get_figure_colors(fig, num_colors=10, data_only=False, print_colors=False)

# Access both data and decoration colors
data_colors = all_colors['hex']['data_colors']
decoration_colors = all_colors['hex']['decoration_colors']
 
print(f"Data colors: {len(data_colors)}")
print(f"Decoration colors: {len(decoration_colors)}")